In [27]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import plotly.express as px
import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    multilabel_confusion_matrix,
)
from catboost.utils import select_threshold
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification,
)
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm

In [28]:

wr.config.s3_endpoint_url = "http://192.168.1.7:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
]
allInputFeature = caseFeatures + allVisionFeatures
targetCol = [
    x for x in multilabelDf.columns if x not in allInputFeature and x != "CaseID"
]
allPredModel = []
allPreds = []
allGt = []

for part in tqdm(targetCol):
    X = multilabelDf[allInputFeature + ["CaseID"]]
    Y = multilabelDf[part].to_frame()
    mulitlearnStratify = StratifiedKFold(n_splits=2)
    allCaseIdByPart = []
    allPredByPart = []
    allGtByPart = []

    for kfoldId, (train_index, test_index) in enumerate(mulitlearnStratify.split(X, Y)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        trainCaseId = X_train["CaseID"]
        testCaseId = X_test["CaseID"]
        X_train = X_train[allInputFeature]
        X_test = X_test[allInputFeature]
        train_pool = Pool(
            X_train, Y_train, cat_features=caseFeatures + allVisionFeatures
        )
        test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)
        clf = CatBoostClassifier(
            loss_function="CrossEntropy",
            eval_metric="F1",
            iterations=20,
            task_type="GPU",
            # class_names=["not_dmg", "dmg"],
        )
        clf.fit(
            train_pool, eval_set=test_pool, metric_period=20, plot=False, verbose=0
        )
        test_predict = clf.predict(X_test)
        b = Y_test.values.squeeze(1).astype(np.int64)
        acc = accuracy_score(b, test_predict)
        confMat = confusion_matrix(b, test_predict)
        pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
        tn = confMat[0][0]
        tp = confMat[1][1]
        fp = confMat[0][1]
        fn = confMat[1][0]
        totalSample = fp + fn + tp + tn
        acc = (tp + tn) / (fp + fn + tp + tn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        assert (tp / (tp + fn)) + (fn / (tp + fn)) == (tn / (tn + fp)) + (
            fp / (tn + fp)
        ) 
        allPredByPart.extend(test_predict)
        
        allCaseIdByPart.extend(testCaseId)
        allGtByPart.extend(b)

        allPredModel.append(
            {   "kfold" : kfoldId,
                "part": part,
                "tp": tp / (tp + fn),
                "tn": tn / (tn + fp),
                "fp": fp / (tn + fp),
                "fn": fn / (tp + fn),
                "acc": acc,
                "precision": precision,
                "recall": recall,
                "f1": (2 * precision * recall) / (precision + recall),
                "pos_count": pos_count,
            }
        )
    allPreds.append({
    "CaseID": allCaseIdByPart,
    part : allPredByPart})
    allGt.append({
    "CaseID": allCaseIdByPart,
    part : allGtByPart})
 

evalMetrics = pd.json_normalize(allPredModel)
avgPrecision = evalMetrics["precision"].mean()
avgRecall = evalMetrics["recall"].mean()
avgF1 = evalMetrics["f1"].mean()
avgTp = evalMetrics["tp"].mean()
avgTn = evalMetrics["tn"].mean()
avgAcc = evalMetrics["acc"].mean()
avgFn = evalMetrics["fn"].mean()

print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Avg TP : {avgTp}")
print(f"Avg TN : {avgTn}")
print(f"Avg FN : {avgFn}")

print(f"avgAccs : {avgAcc}")

 71%|███████   | 24/34 [01:39<00:40,  4.06s/it]/tmp/ipykernel_5771/2655473936.py:58: RuntimeWarning:

invalid value encountered in long_scalars

100%|██████████| 34/34 [02:22<00:00,  4.18s/it]

Avg Precision : 0.875194686639557
Avg Recall : 0.825187217374282
Avg F1 : 0.8060874958174131
Avg TP : 0.825187217374282
Avg TN : 0.9533166888424378
Avg FN : 0.17481278262571798
avgAccs : 0.9342326410647631


In [29]:
evalMetrics = evalMetrics[evalMetrics["kfold"] == 0]

In [30]:
fig = px.bar(evalMetrics, x = 'part', y = 'tp')

fig.add_hline(y=avgTp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [31]:
fig = px.bar(evalMetrics, x = 'part', y = 'tn')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [32]:
fig = px.bar(evalMetrics, x = 'part', y = 'acc')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [33]:
multilabelPred = pd.DataFrame(pd.DataFrame(allPreds[0]))
for p in tqdm(allPreds[1:]):
    dfPart = pd.DataFrame(p)
    multilabelPred = multilabelPred.merge(dfPart, on="CaseID")
multilabelPred.sort_values(by="CaseID", inplace=True)
multilabelPred

100%|██████████| 33/33 [01:00<00:00,  1.84s/it]


,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
147543,10000004,0,1,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
550582,10000008,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
474725,10000012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52174,10000016,1,1,1,1,1,1,0,0,1,...,1,0,1,0,0,0,0,0,0,0
147544,10000021,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215574,13563056,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
514319,13563063,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
215575,13563071,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
447344,13563089,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [34]:
multilabelGt = pd.DataFrame(pd.DataFrame(allGt[0]))
for p in tqdm(allGt[1:]):
    dfPart = pd.DataFrame(p)
    multilabelGt = multilabelGt.merge(dfPart, on="CaseID")
multilabelGt.sort_values(by="CaseID", inplace=True)


100%|██████████| 33/33 [01:00<00:00,  1.84s/it]


In [35]:
multilabelGt = multilabelGt[multilabelPred.columns]
multilabelGt

,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
147543,10000004,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550582,10000008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
474725,10000012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52174,10000016,1,1,1,1,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
147544,10000021,0,1,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215574,13563056,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
514319,13563063,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
215575,13563071,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
447344,13563089,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [36]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list), acc_list

In [37]:
subsetAcc2, acc_list = hamming_score(multilabelGt[targetCol].values, multilabelPred[targetCol].values)
subsetAcc2

0.8481551642995004